In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder \
    .appName("test_notebook") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/05 06:13:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/05 06:13:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/05 06:13:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
df = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option("multiLine", "true") \
    .option("escape", '"') \
    .load('s3a://linh-test/amazon-products.csv')
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: string (nullable = true)
 |-- initial_price: string (nullable = true)
 |-- final_price: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- reviews_count: integer (nullable = true)
 |-- categories: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- buybox_seller: string (nullable = true)
 |-- number_of_sellers: integer (nullable = true)
 |-- root_bs_rank: integer (nullable = true)
 |-- answered_questions: integer (nullable = true)
 |-- domain: string (nullable = true)
 |-- images_count: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- video_count: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- item_weight: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- product_dimensi

In [5]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- description: string (nullable = true)
 |-- initial_price: string (nullable = true)
 |-- final_price: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- reviews_count: integer (nullable = true)
 |-- categories: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- buybox_seller: string (nullable = true)
 |-- number_of_sellers: integer (nullable = true)
 |-- root_bs_rank: integer (nullable = true)
 |-- answered_questions: integer (nullable = true)
 |-- domain: string (nullable = true)
 |-- images_count: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- video_count: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- item_weight: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- product_dimensi

In [8]:
df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.bronze.amazon_products')

In [9]:
spark.stop()